# Preprocessing

## Import

In [23]:
import os
import pathlib
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import mne
import mne_bids
from mne.preprocessing import (ICA, create_eog_epochs, 
                               compute_proj_eog, create_ecg_epochs, corrmap)
                               
matplotlib.use('Qt5Agg')
mne.set_log_level('warning') #All mne functions will only spit out output if they have reason to.

## ICA Analysis

### Template

#### Input Template

In [ ]:
template_raw = mne.io.read_raw(pathlib.Path('out_data') / '02_interpolation_data' / 'sub-01' / 
                      'sub-01_ses-01_task-WM1isi0_interpolated_data.fif')
template_raw.load_data()

#### Pass Template Filter

In [25]:
high_pass_template = template_raw.filter(l_freq=1, h_freq=None) 

#### Template ICA Analysis

In [26]:
template_ica = ICA(n_components=15, max_iter='auto', random_state=97)
template_ica.fit(high_pass_template)
template_ica

Method,fastica
Fit,28 iterations on raw data (361490 samples)
ICA components,15
Explained variance,100.0 %
Available PCA components,127
Channel types,eeg
ICA components marked for exclusion,—


### Data 

#### Input Data

In [ ]:
subname = 'sub-'+subject
filename = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_interpolated_data.fif'
filename

In [ ]:
raw = mne.io.read_raw(pathlib.Path('out_data') / '02_interpolation_data' / subname / 
                      filename)
events, event_id = mne.events_from_annotations(raw) 
raw.load_data() 

#### Pass Data Filter

In [28]:
high_pass_raw = raw.filter(l_freq=1, h_freq=None) 

#### Data ICA Analysis

In [29]:
ica = ICA(n_components=15, max_iter='auto', random_state=97)
ica.fit(high_pass_raw)
ica

Method,fastica
Fit,32 iterations on raw data (486748 samples)
ICA components,15
Explained variance,100.0 %
Available PCA components,127
Channel types,eeg
ICA components marked for exclusion,—


## Eye Blinks Rejection

### Reject Eye Blinks Component

In [31]:
icas = list()
icas.append(template_ica)
icas.append(ica)

In [32]:
corrmap(icas, template=(0, 6), threshold = .9, label = 'eyeblink', plot = True)

(<MNEFigure size 195x260 with 1 Axes>, <MNEFigure size 390x260 with 2 Axes>)

In [33]:
icas[1].exclude = icas[1].labels_['eyeblink']

### Create New Raw Data

In [34]:
reconst_raw = raw.copy()
icas[1].apply(reconst_raw)

<RawBrainVision | sub-08_ses-02_task-WM1isi500_eeg.eeg, 129 x 486748 (486.7 s), ~479.2 MB, data loaded>

In [35]:
reconst_raw.plot()

## Epoch Comparison

### Input Old Epochs

In [39]:
oldepochs = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_epochs-epo.fif'

In [40]:
old_epochs = mne.read_epochs(pathlib.Path('out_data')
            / '03_init_epochs' / subname / oldepochs)
old_epochs

Number of events,527
Events,TsD-: 131WMD+: 132bgin: 132fix+: 132
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


#### Old Epoch Rejection Filter

In [41]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

old_epochs.drop_bad(reject=reject_criteria)

Number of events,19
Events,TsD-: 0WMD+: 7bgin: 0fix+: 12
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


In [42]:
old_epochs

Number of events,19
Events,TsD-: 0WMD+: 7bgin: 0fix+: 12
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


### Run New Epochs

#### Filter Raw Data

In [44]:
filter_raw = reconst_raw.filter(l_freq=0.1, h_freq=40) 
filter_raw

<RawBrainVision | sub-08_ses-02_task-WM1isi500_eeg.eeg, 129 x 486748 (486.7 s), ~479.2 MB, data loaded>

#### Start New Epochs

In [45]:
tmin = -0.25 #300 ms before the event
tmax = 0.8 #500 ms after the onset

new_epochs = mne.Epochs(reconst_raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    preload=True)
new_epochs

Number of events,527
Events,TsD-: 131WMD+: 132bgin: 132fix+: 132
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


#### New Epoch Rejection Filter

In [47]:
reject_criteria = dict(eeg=100e-6) # 100 µV, what should our rejection criteria be? 

new_epochs.drop_bad(reject=reject_criteria)

Number of events,83
Events,TsD-: 8WMD+: 33bgin: 8fix+: 34
Time range,-0.250 – 0.800 sec
Baseline,-0.250 – 0.000 sec


## Result

## Saving Cleaned Epoch Data 

In [19]:
filename = 'sub-'+subject+'_ses-'+session+'_task-'+task+'_cleaned_data.fif'

In [20]:
new_epochs.save(pathlib.Path('out_data')
            / '04_preprocessing' / subname / filename)

C:\Users\trevo\AppData\Local\Temp\ipykernel_14288\900852233.py:1: RuntimeWarning: This filename (out_data\03_preprocessing\sub-01_ses-02_task-WM1isi250_cleaned_data.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  new_epochs.save(pathlib.Path('out_data')
